# General information about various "XGBoost-based" algorithms
Various ML algorithms considered for this challenge, which may help us during the process.

## XGBoost
* Algorithm that has recently been dominating applied machine learning and Kaggle competitions for structured or tabular data (https://machinelearningmastery.com/gentle-introduction-xgboost-applied-machine-learning/)
* Its' a state of the art for structured data problems
* Designed for speed and performance

## LightGBM
* LightGBM https://papers.nips.cc/paper/6907-lightgbm-a-highly-efficient-gradient-boosting-decision-tree.pdf
* LightGBM improves on XGBoost (https://medium.com/kaggle-nyc/gradient-boosting-decision-trees-xgboost-vs-lightgbm-and-catboost-72df6979e0bb)
* LightGBM outperforms XGBoost in training time on CPU and it seems to be the best choice for Kaggle (read more performance information here https://github.com/szilard/GBM-perf)
* It trains fast comapred to similar solutions (Catboost, XGBoost)
* Many notebooks use this LGBM, so believe it's a good way to go
* High speed, can handle large size of data, takes lower memory to run (important for this challenge)
* Sensitive to overfitting, so it’s not advisable to run it on small data (data with 10 000+ rows should be ok)
* Implementation is easy, the problem is parameter tuning (it covers more than 100 parameters, luckily it's kind of important to only know a few of them)

## Catboost
* Catboost a possible improvement as it handles categorical features better, but it can be slower, probably worth into looking if having a lot of categorical features, then the accuracy can be better than with LGBM (https://medium.com/kaggle-nyc/gradient-boosting-decision-trees-xgboost-vs-lightgbm-and-catboost-72df6979e0bb, https://datascience.stackexchange.com/questions/49567/lightgbm-vs-xgboost-vs-catboost, https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/138881)
* As we don't have that many categorical features, probably not worth looking into it

In [ ]:
# More to be explored in this notebook https://www.kaggle.com/ramitjaal/m5-groupkfold-0-47474
# At the end of that notebook is also a list of many relevant and similar notebooks that can be helpful

import pandas as pd
import numpy as np
import dask.dataframe as dd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb # our star
from sklearn import preprocessing, metrics
import gc # cleaning up the mess
import joblib # running Python functions as pipeline jobs
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Source: https://www.kaggle.com/ragnar123/very-fst-model
# Pretty much every public LGBM notebook is using this memory reduction function

# Just a reducing memory function
# Return a reduced dataframe
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
# Read files, reduce memory of all of the four input files (by calling the reduce_mem_usage function)
# Return the input files reduced
INPUT_DIR_PATH = '../input/m5-forecasting-accuracy/'

def read_data():
    sell_prices_df = pd.read_csv(INPUT_DIR_PATH + 'sell_prices.csv')
    sell_prices_df = reduce_mem_usage(sell_prices_df)
    print('Sell prices has {} rows and {} columns'.format(sell_prices_df.shape[0], sell_prices_df.shape[1]))

    calendar_df = pd.read_csv(INPUT_DIR_PATH + 'calendar.csv')
    calendar_df = reduce_mem_usage(calendar_df)
    print('Calendar has {} rows and {} columns'.format(calendar_df.shape[0], calendar_df.shape[1]))

    sales_train_validation_df = pd.read_csv(INPUT_DIR_PATH + 'sales_train_validation.csv')
    print('Sales train validation has {} rows and {} columns'.format(sales_train_validation_df.shape[0], sales_train_validation_df.shape[1]))

    submission_df = pd.read_csv(INPUT_DIR_PATH + 'sample_submission.csv')
    return sell_prices_df, calendar_df, sales_train_validation_df, submission_df

In [ ]:
# Get the reduced files
# At this point we just reduced size of the input files
# The files are all separate, no melting or merging has been done yet
sell_prices_df, calendar_df, sales_train_validation_df, submission_df = read_data()

In [ ]:
# numeber of items
NUM_ITEMS = 30490

# number of days to predict
DAYS_PRED = 28

nrows = 365 * 2 * NUM_ITEMS

In [ ]:
# Encode categorical features
def encode_categorical(df, cols):
    for col in cols:
        # Leave NaN as it is
        le = preprocessing.LabelEncoder()
        not_null = df[col][df[col].notnull()]
        df[col] = pd.Series(le.fit_transform(not_null), index=not_null.index)

    return df


calendar_df = encode_categorical(calendar_df, ["event_name_1", "event_type_1", "event_name_2", "event_type_2"]).pipe(reduce_mem_usage)
sales_train_validation_df = encode_categorical(sales_train_validation_df, ["item_id", "dept_id", "cat_id", "store_id", "state_id"]).pipe(reduce_mem_usage)
sell_prices_df = encode_categorical(sell_prices_df, ["item_id", "store_id"]).pipe(reduce_mem_usage)

In [ ]:
# Source: https://www.kaggle.com/rohitsingh9990/m5-lgbm-fe/data
# nrow set to 55000000 - it's the maximum number of rows Kaggle can handle

# Transform the sales dataframe from a wide format into a long format, then merge all those three files we have
def melt_and_merge(calendar, sell_prices, sales_train_validation, submission, nrows = 55000000, merge = False):
    
    # melt sales data, get it ready for training
    sales_train_validation = pd.melt(sales_train_validation, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    print('Melted sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    sales_train_validation = reduce_mem_usage(sales_train_validation)
    
    sales_train_validation = sales_train_validation.iloc[-nrows:,:]
    
    
    # seperate test dataframes
    test1_rows = [row for row in submission['id'] if 'validation' in row]
    test2_rows = [row for row in submission['id'] if 'evaluation' in row]
    test1 = submission[submission['id'].isin(test1_rows)]
    test2 = submission[submission['id'].isin(test2_rows)]
    
    # change column names
    test1.columns = ['id', 'd_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919', 'd_1920', 'd_1921', 'd_1922', 'd_1923', 'd_1924', 'd_1925', 'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 'd_1931', 
                      'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939', 'd_1940', 'd_1941']
    test2.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949', 'd_1950', 'd_1951', 'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 'd_1959', 
                      'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 'd_1968', 'd_1969']
    
    # get product table
    product = sales_train_validation[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()
    
    # merge with product table
    test2['id'] = test2['id'].str.replace('_evaluation','_validation')
    test1 = test1.merge(product, how = 'left', on = 'id')
    test2 = test2.merge(product, how = 'left', on = 'id')
    test2['id'] = test2['id'].str.replace('_validation','_evaluation')
    
    test1 = pd.melt(test1, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    test2 = pd.melt(test2, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
    
    sales_train_validation['part'] = 'train'
    test1['part'] = 'test1'
    test2['part'] = 'test2'
    
    data = pd.concat([sales_train_validation, test1, test2], axis = 0)
    
    del sales_train_validation, test1, test2
    
    print(data.shape) # check
    
    # delete test2 for now
    data = data[data['part'] != 'test2']
    
    if merge:
        data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
        data.drop(['d', 'day'], inplace = True, axis = 1)
        data = data.merge(sell_prices, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')
        print('Our final dataset to train has {} rows and {} columns'.format(data.shape[0], data.shape[1]))
    else: 
        pass
    
    gc.collect()
    
    return data

In [ ]:
nrows = 27500000
data = melt_and_merge(calendar_df, sell_prices_df, sales_train_validation_df, submission_df, nrows = nrows, merge = True)

In [ ]:
# Now we have data melted together in one table, let's do some FE magic with it
data.head()

In [ ]:
# Give NaN categorical features values
def transform1(data):
    nan_features = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2','sell_price']
    for feature in nan_features:
        data[feature].fillna('unknown', inplace = True)
        
    return data

In [ ]:
data = transform1(data)
data = reduce_mem_usage(data)

In [ ]:
data.head()

In [ ]:
# Label-encode our categorical features
def transform2(data):
    cat = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'weekday', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for feature in cat:
        # we have to represent the strings as numerical values
        encoder = preprocessing.LabelEncoder() # Encode target labels with value between 0 and n_classes-1.
        print(feature)
        data[feature] = encoder.fit_transform(data[feature].astype(str)) # Fit to data, then transform it.
    
    return data

In [ ]:
data = transform2(data)

In [ ]:
data = reduce_mem_usage(data)

In [ ]:
data.head()

In [ ]:
# Remove rows with unknown sell_price and convert sell_price column from 'object' data type to 'float64'
data.drop(data[data.sell_price == 'unknown'].index, inplace=True)
data["sell_price"] = data.sell_price.astype(float)

In [ ]:
data.head()

In [ ]:
# test rolling demand fe
# data_backup = data
# data = data_backup

# Transform the date feature
data['date'] = pd.to_datetime(data.date)

In [ ]:
# Feature Engineering

# Notebooks with some cool features:
# https://www.kaggle.com/albertobc/m5-feature-engineering-rolling-demand-features
# https://www.kaggle.com/albertobc/m5-nuevos-par-metros-para-el-lgbm
# https://www.kaggle.com/nxrprime/preprocessing-fe
# Times, rolling means, demand lags, max. min. sales etc.

# Rolling mean - average of demand in a given time-span
# Lag features - values at prior time steps

def fe(data):

    # Time features
    # 'weekday' already encoded in the labelEncoder
    data['date'] = pd.to_datetime(data['date'])
    data['wday'] = data['date'].dt.day
    data['month'] = data['date'].dt.month
    data['year'] = data['date'].dt.year

    # rolling demand features
    data['lag_t28'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
    data['lag_t1'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(29))
    data['lag_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(30))
    data['rolling_mean_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).mean())
    data['rolling_mean_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).mean())
    data['rolling_mean_t60'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(60).mean())
    data['rolling_mean_t90'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(90).mean())
    data['rolling_mean_t180'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(180).mean())
    
    # Skewness and Kurtosis
    # Along with skewness, kurtosis is an important descriptive statistic of data distribution.
    
    # Skewness is a measure of the asymmetry of the probability distribution of a real-valued random variable about its mean
    data['rolling_skew_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).skew())
    
    # Kurtosis is a statistical measure that defines how heavily the tails of a distribution differ from the tails of a normal distribution. 
    # In other words, kurtosis identifies whether the tails of a given distribution contain extreme values.
    data['rolling_kurt_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).kurt())
    
    return data

In [ ]:
data = fe(data)
data = reduce_mem_usage(data)

In [ ]:
data.head()

In [ ]:
# Going to evaluate with the last 28 days
x_train = data[data['date'] <= '2016-03-27']
y_train = x_train['demand']
x_val = data[(data['date'] > '2016-03-27') & (data['date'] <= '2016-04-24')]
y_val = x_val['demand']
test = data[(data['date'] > '2016-04-24')]
del data
gc.collect()

In [ ]:
# Use RMSE as a proxy for WRMSSE
# The lower the better (currently having 2.15 for our best solution, though sub 2.00 is definitelly possible)
# There's a correlation between RMSE and WRMSSE, but maybe will cause overfitting for the private LB (Leaderboard)
# (overfitting happens terribly lot in Kaggle competitions)
# Following some discussions, it clearly states that e.g. having 2.10 local RMSE can eventually produce worse
# results for LB than 2.15 local RMSE
# So we can use it as a proxy, but it's not "the ultimate" tell

param = {
    "objective" : "poisson", # achieves good empirical results
    "metric" : "rmse", # proxy for wrmsse
    "force_row_wise" : True,
    "learning_rate" : 0.075, # 0.1 and 0.05 both performed worse 
    "sub_row" : 0.75,
    "bagging_freq" : 1,
    "lambda_l2" : 0.1,
    "nthread" : 4,
    "verbosity": 1,
    "num_iterations" : 1200, # different numbers also performed worse
    "num_leaves": 128,
    "min_data_in_leaf": 100}


# Drop some columns which we don't want to have as features
feature_cols = x_train.columns.drop(['id','part','date','demand'])
print(feature_cols)

train_set = lgb.Dataset(x_train[feature_cols], y_train)
val_set = lgb.Dataset(x_val[feature_cols], y_val)
    
del x_train, y_train

# Let's train!
bst = lgb.train(param, train_set, valid_sets=[train_set, val_set], verbose_eval=20)

In [ ]:
joblib.dump(bst, 'lgbm_0.sav')

val_pred = bst.predict(x_val[feature_cols], num_iteration=bst.best_iteration)
val_score = np.sqrt(metrics.mean_squared_error(val_pred, y_val))
print(f'Our val rmse score is {val_score}')
y_pred = bst.predict(test[feature_cols], num_iteration=bst.best_iteration)
test['demand'] = y_pred

In [ ]:
def predict(test, submission):
    predictions = test[['id', 'date', 'demand']]
    predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'demand').reset_index()
    predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

    evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
    evaluation = submission[submission['id'].isin(evaluation_rows)]

    validation = submission[['id']].merge(predictions, on = 'id')
    final = pd.concat([validation, evaluation])
    final.to_csv('submission.csv', index = False)

In [ ]:
predict(test, submission_df)

In [ ]:
output = pd.read_csv('submission.csv')
print(output)